In [119]:
import numpy as np
import pandas as pd
import math
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import spsolve

def printVar (varname, var):

    print(varname)
    print(var)

def saveArray (name, array):

    pd.DataFrame(array).to_csv("savedVariables/" + name + ".csv")



In [120]:
# Solve the 2D Navier-Cauchy equation using a segregated finite volume method

# Define mesh geometry and boundary conditions

Lx = .01 #width of domain
Ly = .01 #height of domain

fac = 1

u_left = fac*5e-6      #u boundary condition at the left boundary
v_left = fac*3e-6      #v boundary condition at the left boundary

u_right = fac*5e-6     #u boundary condition at the right boundary
v_right = fac*1e-6     #v boundary condition at the right boundary

u_top = fac*1e-6       #u boundary condition at the top boundary
v_top = fac*1e-6       #v boundary condition at the top boundary

vubottom = fac*7e-6    #u boundary condition at the bottom boundary 
v_bottom = fac*1e-6    #v boundary condition at the bottom boundary

nx = 8         #number of control volumes in the x-direction
ny = 8            #number of control volumes in the y-direction


In [121]:
# Solve the 2D Navier-Cauchy equation using a segregated finite volume method

# Define mesh geometry and boundary conditions

Lx = .01 #width of domain
Ly = .01 #height of domain

fac = 1000000

tr_left_x = -1e-6*fac      #u boundary condition at the left boundary
tr_left_y = -1e-6*fac     #v boundary condition at the left boundary

tr_right_x = 1e-6*fac    #u boundary condition at the right boundary
tr_right_y = 1e-6*fac     #v boundary condition at the right boundary

tr_top_x = 1e-6*fac       #u boundary condition at the top boundary
tr_top_y = 1e-6*fac       #v boundary condition at the top boundary

tr_bottom_x = -1e-6*fac    #u boundary condition at the bottom boundary 
tr_bottom_y = -1e-6*fac    #v boundary condition at the bottom boundary

nx = 8            #number of control volumes in the x-direction
ny = 8            #number of control volumes in the y-direction


In [122]:
# Solve the 2D Navier-Cauchy equation using a segregated finite volume method

# Define mesh geometry and boundary conditions

# Lx = .01 #width of domain
# Ly = .01 #height of domain

# tr_left = 0e-6      #u boundary condition at the left boundary
# v_left = 0e-6      #v boundary condition at the left boundary

# u_right = 0e-6     #u boundary condition at the right boundary
# v_right = 0e-6     #v boundary condition at the right boundary

# u_top = 0e-6       #u boundary condition at the top boundary
# v_top = 0e-6       #v boundary condition at the top boundary

# u_bottom = 0e-6    #u boundary condition at the bottom boundary 
# v_bottom = 0e-6    #v boundary condition at the bottom boundary

# nx = 10            #number of control volumes in the x-direction
# ny = 10            #number of control volumes in the y-direction


In [123]:
# Define material properties

rho = 8050         #density of steel in kg/m^3
mu = 78*10e9       #shear modulus in Pa
K = 1.6*10e11      #bulk modulus in Pa

Lambda = K - (2/3)*mu

#print(Lambda)

In [124]:
## Define required variables

tf = 0.01              #total time in seconds
dx = Lx/nx          #length of each control volume in the x-direction
dy = Ly/ny          #length of each control volume in the y-direction
dt = tf/10         #size of time steps
Sfx = dy            #area vector x component (Area of East and West Faces)
Sfy = dx            #area vector y component (Area of North and South Faces)


In [125]:
## Define  position and time vectors

t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
x = np.zeros((1,nx+2))   #position vector defining the center points of the control volumes in the x-direction.
x[0,nx+1] = Lx
x[0,1:nx+1] = np.arange(dx/2,Lx,dx)

t = np.array(np.arange(0,tf, dt))        #time vector (could possibly use a list)
y = np.zeros((1,ny+2))   #position vector defining the center points of the control volumes in the y-direction.
y[0,ny+1] = Ly
y[0,1:ny+1] = np.arange(dy/2,Ly,dy)

#print(x)
#print(y)


In [126]:
## Define required displacement matrices and b vector

u_new = np.zeros([(ny+2)*(nx+2),2])         #unknown displacements at t + 1    (Column 1 = x, Column2 = y)
u_old = np.zeros([(ny+2)*(nx+2),2])         #displacement at time t
u_old_old = np.zeros([(ny+2)*(nx+2),2])     #displacement at time t - 1

#rhs vector
b_x = np.zeros([(ny+2)*(nx+2),1])
b_y = np.zeros([(ny+2)*(nx+2),1])

In [127]:
## Define coefficient matrix A with no boundary conditions

A_no_boundary = np.zeros([(ny+2)*(nx+2), (ny+2)*(nx+2)])
A_no_boundary = csr_matrix(A_no_boundary)

# To display A_no_boundary:

#print(A_no_boundary.todense().shape)

## X equation:

In [128]:
# Only for x-equation:

a_P_x = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfx)/dx + ((2*mu)*Sfy)/dy

a_N_x = ((mu)*Sfy)/dy
a_S_x = ((mu)*Sfy)/dy
a_E_x = ((2*mu + Lambda)*Sfx)/dx
a_W_x = ((2*mu + Lambda)*Sfx)/dx


In [129]:
A_no_boundary_x = A_no_boundary

## Define internal coefficients of matrix A for values with no B.C.s for x-equation

#j is column number in mesh (y-coord)
#k is point number in mesh

for j in np.arange(2, ny):     #main diagonal, starting from y=1 because all values at y=0 have B.C.s

    for k in np.arange((j*(nx+2)+2), (j*(nx+2)+nx)):  
        
        A_no_boundary_x[k,k] = a_P_x
        A_no_boundary_x[k,k+nx+2] = -a_N_x
        A_no_boundary_x[k,k-nx-2] = -a_S_x
        A_no_boundary_x[k,k+1] = -a_E_x
        A_no_boundary_x[k,k-1] = -a_W_x

        # #print("\tN: " + str(k+nx), "\nW: " + str(k-1), "\tP: " + str(k), "\tE: " + str(k+1), "\n\tS: " + str(k-nx) + "\n")

saveArray("A_no_boundary_x", A_no_boundary_x.todense())

## y equation:

In [130]:
# Only for y-equation:

a_P_y = (rho*dx*dy/(dt**2))+((4*mu+2*Lambda)*Sfy)/dy + ((2*mu)*Sfx)/dx 

a_N_y = ((2*mu + Lambda)*Sfy)/dy
a_S_y = ((2*mu + Lambda)*Sfy)/dy
a_E_y = ((mu)*Sfx)/dx
a_W_y = ((mu)*Sfx)/dx


In [131]:
A_no_boundary_y = A_no_boundary

## Define internal coefficients of matrix A for values with no B.C.s for x-equation

#j is column number in mesh (y-coord)
#k is point number in mesh

for j in np.arange(2, ny):     #main diagonal, starting from y=1 because all values at y=0 have B.C.s

    for k in np.arange((j*(nx+2)+2), (j*(nx+2)+nx)):  
        
        A_no_boundary_y[k,k] = a_P_y

        A_no_boundary_y[k,k+nx+2] = -a_N_y
        A_no_boundary_y[k,k-nx-2] = -a_S_y
        A_no_boundary_y[k,k+1] = -a_E_y
        A_no_boundary_y[k,k-1] = -a_W_y

        # #print("\tN: " + str(k+nx), "\nW: " + str(k-1), "\tP: " + str(k), "\tE: " + str(k+1), "\n\tS: " + str(k-nx) + "\n")

In [132]:
## Time loop

!rm -r savedVariables
!mkdir savedVariables

for time in [1]:

    #print("\nTime = " + str(time))
    
    # Set number of maximum iterations for convergence
    
    icorr = 1
    maxcorr = 100

    # Make directory

    !mkdir savedVariables/{time}
    
    while True:
    # for iteration in [1]:
            
        u_previous = u_new #store latest guess values
        
        # Prepare matrix A and b vector for the x-equation
        
        A_x = A_no_boundary_x #make copy of matrix A with no boundary conditions
        
        for k in np.arange(0,(nx+2)*(ny+2)):   # k is the cell number

            # Make indexing variables    # Could change this to a class/objects?
            # if ( k > nx+2 ) & (k < (nx+2)*(ny+2) - (nx+2) -2): # greater than bottom left corner and less than top right

            if (k < nx +2 ) & (k > 0): 

                U_N = u_previous[k+(nx+2),0]
                V_N = u_previous[k+(nx+2),1]
                U_E = u_previous[k+1,0]
                V_E = u_previous[k+1,1]
                U_W = u_previous[k-1,0]
                V_W = u_previous[k-1,1]
                U_NE = u_previous[k+(nx+2)+1,0]
                V_NE = u_previous[k+(nx+2)+1,1]
                U_NW = u_previous[k+(nx+2)-1,0]
                V_NW = u_previous[k+(nx+2)-1,1]
            
            elif(k < (nx+2)*(ny+2) - (nx+2) - 1):
               
                U_N = u_previous[k+(nx+2),0]
                V_N = u_previous[k+(nx+2),1]
                U_S = u_previous[k-(nx+2),0]
                V_S = u_previous[k-(nx+2),1]
                U_E = u_previous[k+1,0]
                V_E = u_previous[k+1,1]
                U_W = u_previous[k-1,0]
                V_W = u_previous[k-1,1]
                U_NE = u_previous[k+(nx+2)+1,0]
                V_NE = u_previous[k+(nx+2)+1,1]
                U_NW = u_previous[k+(nx+2)-1,0]
                V_NW = u_previous[k+(nx+2)-1,1]
                U_SE = u_previous[k-(nx+2)+1,0]
                V_SE = u_previous[k-(nx+2)+1,1]
                U_SW = u_previous[k-(nx+2)-1,0]
                V_SW = u_previous[k-(nx+2)-1,1]

            elif(k < (nx+2)*(ny+2) - 2):

                U_S = u_previous[k-(nx+2),0]
                V_S = u_previous[k-(nx+2),1]
                U_E = u_previous[k+1,0]
                V_E = u_previous[k+1,1]
                U_W = u_previous[k-1,0]
                V_W = u_previous[k-1,1]
                U_SE = u_previous[k-(nx+2)+1,0]
                V_SE = u_previous[k-(nx+2)+1,1]
                U_SW = u_previous[k-(nx+2)-1,0]
                V_SW = u_previous[k-(nx+2)-1,1]

            #Bottom left corner coefficients
                
            if k == nx+3:

                #print(str(k) + ": Bottom left corner coefficients")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)

                #an
                A_x[k,k+nx+2] = -(Sfy/dy)*mu

                #ae
                A_x[k,k+1] = - (Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                + V_NE
                                                + V_E
                                                - V_NW #l
                                                - V_W #l
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    + V_NE
                                                    + V_N
                                                    - V_SE #b
                                                    - V_S #b
                                                )
                            +
                            Sfy*tr_bottom_x
                            +
                            Sfx*tr_left_x
                        )   

            #Bottom right corner coefficients
                
            elif  k == 2*(nx+2) -2:

                #print(str(k) + ": Bottom right corner coefficients")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)

                #an
                A_x[k,k+nx+2] = -(Sfy/dy)*mu

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                #b_x = A + B(u_b) + C(u_r) + D(-v_w -v_nw + 2v_r) + E(-v_n -v_nw + 2v_b)


                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                + V_NE #r
                                                + V_E #r
                                                - V_NW
                                                - V_W
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    - V_N
                                                    - V_NW
                                                    + V_S #b
                                                    + V_SW #b
                                                )
                            +
                            Sfx*tr_right_x
                            +
                            Sfy*tr_bottom_x
                        ) 

            # Top Left Corner Coeficients
                
            elif  k == (nx+2)*(ny+2) - 2*(nx+2) +1:

                #print(str(k) + ":Top Left corner coefficients")

                #ap                
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)
                
                #as
                A_x[k,k-nx-2] = -(Sfy/dy)*mu

                #ae                
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)
                
                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                - V_E
                                                - V_SE
                                                + V_W #l
                                                + V_SW #l
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    + V_NE #t
                                                    + V_N #t
                                                    - V_SE
                                                    - V_S
                                                )
                            +
                            Sfy*tr_top_x
                            +
                            Sfx*tr_left_x
                        )
                
            # Top right corner coefficients
                
            elif  k == (nx+2)*(ny+2) - (nx+2) -2:   

                #print(str(k) + ":Top right corner coefficients")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)

                #as
                A_x[k,k-nx-2] = -(Sfy/dy)*mu

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                - V_E #b
                                                - V_SE #b
                                                + V_W
                                                + V_SW
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    - V_N #t
                                                    - V_NW #t
                                                    + V_S
                                                    + V_SW
                                                )
                            +
                            Sfy*tr_top_x
                            +
                            Sfx*tr_right_x
                        )
                
            # Center Bottom Boundaries

            elif  (k > nx+3) & (k < 2*(nx+2) -1):

                # #print(k)

                #print(str(k) + ": Bottom Center Coefficents for Cell")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda) + (Sfx/dx)*(2*mu + Lambda)

                #an
                A_x[k,k+nx+2] = -(Sfy/dy)*mu
                
                #ae
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(4*dx))*mu*(
                                                + V_NE
                                                + V_E
                                                - V_NW
                                                - V_W
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    + V_NE
                                                    + V_N
                                                    - V_SE #b
                                                    - V_S #b
                                                )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    - V_N
                                                    - V_NW
                                                    + V_S #b
                                                    + V_SW #b
                                                )
                            +
                            Sfy*tr_bottom_x
                        )                                 

            # Center Top Boundaries

            elif  ( k > (nx+2)*(ny+2) - 2*(nx+2) +1 ) & ( k < (nx+2)*(ny+2) - (nx+2) -2 ):

                # #print(k)

                #print(str(k) + ": Top Center Coefficents for Cell")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda) + (Sfx/dx)*(2*mu + Lambda)

                #as
                A_x[k,k-nx-2] = -(Sfy/dy)*mu
                
                #ae
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(4*dx))*mu*(
                                                - V_E
                                                - V_SE
                                                + V_W
                                                + V_SW
                                            )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    + V_NE #t
                                                    + V_N #t
                                                    - V_SE
                                                    - V_S
                                                )
                            +
                            (Sfx/(3*dy))*Lambda*(
                                                    - V_N #t
                                                    - V_NW #t
                                                    + V_S
                                                    + V_SW
                                                )
                            +
                            Sfy*tr_top_x
                        )
            
            # Center Left Boundaries

            elif  ( k > nx+3) & ( k%(nx+2) == 1 ) & ( k < (nx+2)*(ny+2) - 2*(nx+2) +1):

                #print(str(k) + ": Left Center Coefficents for Cell")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)

                #an
                A_x[k,k+nx+2] = -(Sfy/dy)*mu
                
                #as
                A_x[k,k-nx-2] = -(Sfy/dy)*mu

                #ae
                A_x[k,k+1] = -(Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                + V_NE
                                                + V_E
                                                - V_NW #l
                                                - V_W #l
                                            )
                            +
                            (Sfy/(3*dx))*mu*(
                                                - V_E
                                                - V_SE
                                                + V_W #l
                                                + V_SW #l 
                                            )
                            +
                            (Sfx/(4*dy))*Lambda*(
                                                   + V_NE
                                                   + V_N
                                                   - V_SE
                                                   - V_S
                                                )
                            +
                            Sfx*tr_left_x
                        )

            # Center Right Boundaries

            elif  ( k > 2*(nx+2) -2) & ( k%(nx+2) == nx ) & ( k < (nx+2)*(ny+2) - (nx+2) -2 ):

                #print(str(k) + ": Right Center Coefficents for Cell")

                #ap
                A_x[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*mu + (Sfy/dy)*mu + (Sfx/dx)*(2*mu + Lambda)

                #an
                A_x[k,k+nx+2] = -(Sfy/dy)*mu
                
                #as
                A_x[k,k-nx-2] = -(Sfy/dy)*mu

                #aw
                A_x[k,k-1] = -(Sfx/dx)*(2*mu + Lambda)

                b_x[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy) 
                            +
                            (Sfy/(3*dx))*mu*(
                                                + V_NE #r
                                                + V_E #r
                                                - V_NW 
                                                - V_W 
                                            )
                            +
                            (Sfy/(3*dx))*mu*(
                                                - V_E #r
                                                - V_SE #r
                                                + V_W 
                                                + V_SW 
                                            )
                            +
                            (Sfx/(4*dy))*Lambda*(
                                                   - V_N 
                                                   - V_NW
                                                   + V_S
                                                   + V_SW
                                                )
                            +
                            Sfx*tr_right_x
                        )

            #### Points on Boundary ####

            # Center Bottom

            elif (k > 1) & (k < nx):

                #print(str(k) +  ": Bottom points")

                #ap
                A_x[k,k] = 2*mu/dy

                #an
                A_x[k,k+nx+2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_bottom_x
                        +
                        (mu/(2*dx))*(
                                        + V_E
                                        - V_W
                                    )
                        ) 

            # Center Top

            elif ( k >= ((nx+2)*(ny+2) - nx) ) & ( k < (nx+2)*(ny+2) - 2) :
                
                #print( str(k) + ": Center points" )
                
                #ap
                A_x[k,k] = 2*mu/dy

                #as
                A_x[k,k-nx-2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_top_x
                        +
                        (mu/(2*dx))*(
                                        - V_E
                                        + V_W
                                    )
                        ) 

                
            # Center Left

            elif ( k > 2*(nx+2) -1) & (k < (ny)*(nx+2) ) & (k%(nx+2) == 0):

                #print( str(k) + ": Left points" )

                #ap
                A_x[k,k] = (4*mu + 2*Lambda)/dx

                #ae
                A_x[k,k+1] = -(4*mu + 2*Lambda)/dx

                #b
                b_x[k] =(
                        tr_left_x
                        +
                        (Lambda/(2*dy))*(
                                        + V_N
                                        - V_S
                                    )
                        )

            # Center Right

            elif ( k > 2*(nx+2) -1) & (k < (ny)*(nx+2)) & (k%(nx+2) == nx +1):

                #print( str(k) + ": Right points" )

                #ap
                A_x[k,k] = (4*mu + 2*Lambda)/dx

                #aw
                A_x[k,k-1] = -(4*mu + 2*Lambda)/dx

                #b
                b_x[k] =(
                        tr_right_x
                        +
                        (Lambda/(2*dy))*(
                                        - V_N
                                        + V_S
                                    )
                        )

            # Bottom Left: Bottom

            elif (k == 1):
                #print( str(k) + ": Bottom Left: Bottom points" )
                #ap
                A_x[k,k] = 2*mu/dy

                #an
                A_x[k,k+nx+2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_bottom_x
                        +
                        ((2*mu)/(3*dx))*(
                                        + V_E
                                        - V_W
                                    )
                        ) 

            # Bottom Left: Left

            elif (k == nx + 2):

                #print( str(k) +  ": Bottom Left: Left points" )
                
                #ap
                A_x[k,k] = (4*mu + 2*Lambda)/dx

                #ae
                A_x[k,k+1] = -(4*mu + 2*Lambda)/dx

                #b
                b_x[k] =(
                        tr_left_x
                        +
                        ((2*Lambda)/(3*dy))*(
                                        + V_N
                                        - V_S
                                    )
                        )
            
            # Bottom Right: Right

            elif ( k == 2*(nx+2) -1) :

                #print( str(k) + ": Bottom Right: Right points" )

                #ap
                A_x[k,k] = (4*mu + 2*Lambda)/dx

                #aw
                A_x[k,k-1] = -(4*mu + 2*Lambda)/dx

                #b
                b_x[k] =(
                        tr_right_x
                        +
                        ((2*Lambda)/(3*dy))*(
                                        - V_N
                                        + V_S
                                    )
                        )

            # Bottom Right: Bottom

            elif (k == nx) : 

                #print( str(k) + ": Bottom Right: Bottom points" )


                #ap
                A_x[k,k] = 2*mu/dy

                #an
                A_x[k,k+nx+2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_bottom_x
                        +
                        ((2*mu)/(3*dx))*(
                                        + V_E
                                        - V_W
                                    )
                        )

            # Top Left: Left

            elif ( k == (nx+2)*(ny+2) - 2*(nx+2) ) :

                #print( str(k) + " Top Left: Left points" )

                
                #ap
                A_x[k,k] = (4*mu + 2*Lambda)/dx

                #ae
                A_x[k,k+1] = -(4*mu + 2*Lambda)/dx

                #b
                b_x[k] =(
                        tr_left_x
                        +
                        ((2*Lambda)/(3*dy))*(
                                        + V_N
                                        - V_S
                                    )
                        )

            # Top Left: Top

            elif (k == (nx+2)*(ny+2) - (nx+2) +1):

                #print( str(k) + " Top Left: Top points" )

                
                #ap
                A_x[k,k] = 2*mu/dy

                #as
                A_x[k,k-nx-2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_top_x
                        +
                        ((2*mu)/(3*dx))*(
                                        - V_E
                                        + V_W
                                    )
                        )

            # Top Right: Right

            elif ( k == (nx+2)*(ny+2) - (nx+2) - 1) :

                #print( str(k) + " Top Right: Right points" )

                #ap
                A_x[k,k] = (4*mu + 2*Lambda)/dx

                #aw
                A_x[k,k-1] = -(4*mu + 2*Lambda)/dx

                #b
                b_x[k] =(
                        tr_right_x
                        +
                        ((2*Lambda)/(3*dy))*(
                                        - V_N
                                        + V_S
                                    )
                        )

            # Top Right: Top
            
            elif ( k == (nx+2)*(ny+2) - 2 ) : 

                #print( str(k) + " Top Right: Top points" )

                #ap
                A_x[k,k] = 2*mu/dy

                #as
                A_x[k,k-nx-2] = -2*mu/dy

                #b
                b_x[k] =(
                        tr_top_x
                        +
                        ((2*mu)/(3*dx))*(
                                        - V_E
                                        + V_W
                                    )
                        ) 
            # Corner Points

            elif (k == 0):
                #print(str(k) + ": Bottom Left Corner Point on Boundary")

                A_x[k,k] = 1
                b_x[k] = 1e-20

            elif (k == nx + 1):
                #print(str(k) + ": Bottom Right Corner Point on Boundary")
                A_x[k,k] = 1
                b_x[k] = 1e-20

            elif (k == (nx+2)*(ny +1) ): 
                #print(str(k) + ": Top Left Corner Point on Boundary")
                A_x[k,k] = 1
                b_x[k] = 1e-20

            elif (k == (nx+2)*(ny +2) - 1) : 
                #print(str(k) + ": Top Right Corner Point on Boundary")
                A_x[k,k] = 1
                b_x[k] = 1e-20
                
            else:

                #print(str(k) + ": NO BC Cell")

                b_x[k] = (
                            (rho/(dt**2))*( 2*(u_old[k,0])*dx*dy - u_old_old[k,0]*dx*dy)
                            + (Sfy/(4*dx))*mu*(
                                                + V_NE    # + v_ne
                                                - V_NW    # - v_nw
                                                - V_SE    # - v_se
                                                + V_SW    # + v_sw
                                            )
                            +(Sfx/(4*dy))*Lambda*(
                                                    + V_NE    # + v_ne
                                                    - V_NW    # - v_nw
                                                    - V_SE    # - v_se
                                                    + V_SW    # + v_sw 
                                                )
                            )

        saveArray(str(time) + "/A_x_" + str(icorr) , A_x.todense())   
        saveArray(str(time) + "/b_x_" + str(icorr), b_x)     
        
        ## Solve the x-equation
        
        Ux = spsolve(A_x, b_x)
        Ux = np.array(Ux)

        # #print(Ux)

        saveArray(str(time) + "/Ux_" + str(icorr), Ux)
        
        ## Prepare matrix A and b vector for the y-equation
        
        A_y = A_no_boundary_y #make copy of A with no boundary conditions
        
        for k in np.arange(0,(nx+2)*(ny+2)):   # j is the cell number

            # Make indexing variables    # Could change this to a class/objects?
            if (k < nx +2 ) & (k > 0): 

                U_N = u_previous[k+(nx+2),0]
                V_N = u_previous[k+(nx+2),1]
                U_E = u_previous[k+1,0]
                V_E = u_previous[k+1,1]
                U_W = u_previous[k-1,0]
                V_W = u_previous[k-1,1]
                U_NE = u_previous[k+(nx+2)+1,0]
                V_NE = u_previous[k+(nx+2)+1,1]
                U_NW = u_previous[k+(nx+2)-1,0]
                V_NW = u_previous[k+(nx+2)-1,1]
            
            elif(k < (nx+2)*(ny+2) - (nx+2) - 1):
               
                U_N = u_previous[k+(nx+2),0]
                V_N = u_previous[k+(nx+2),1]
                U_S = u_previous[k-(nx+2),0]
                V_S = u_previous[k-(nx+2),1]
                U_E = u_previous[k+1,0]
                V_E = u_previous[k+1,1]
                U_W = u_previous[k-1,0]
                V_W = u_previous[k-1,1]
                U_NE = u_previous[k+(nx+2)+1,0]
                V_NE = u_previous[k+(nx+2)+1,1]
                U_NW = u_previous[k+(nx+2)-1,0]
                V_NW = u_previous[k+(nx+2)-1,1]
                U_SE = u_previous[k-(nx+2)+1,0]
                V_SE = u_previous[k-(nx+2)+1,1]
                U_SW = u_previous[k-(nx+2)-1,0]
                V_SW = u_previous[k-(nx+2)-1,1]

            elif(k < (nx+2)*(ny+2) - 2):

                U_S = u_previous[k-(nx+2),0]
                V_S = u_previous[k-(nx+2),1]
                U_E = u_previous[k+1,0]
                V_E = u_previous[k+1,1]
                U_W = u_previous[k-1,0]
                V_W = u_previous[k-1,1]
                U_SE = u_previous[k-(nx+2)+1,0]
                V_SE = u_previous[k-(nx+2)+1,1]
                U_SW = u_previous[k-(nx+2)-1,0]
                V_SW = u_previous[k-(nx+2)-1,1]
 
            #Bottom left corner coefficients
            
            if k == nx+3:

                #print(str(k) + ": Bottom left corner coefficients")
                
                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu)    
                
                #an
                A_y[k,k+nx+2] = -(Sfy/dy)*(2*mu + Lambda)

                #ae                
                A_y[k,k+1] = -(Sfx/dx)*(mu)
                
                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                + U_NE
                                                + U_E
                                                - U_NW #l
                                                - U_W #l
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    + U_NE
                                                    + U_N
                                                    - U_SE #b
                                                    - U_S #b
                                                )
                            +
                            Sfy*tr_bottom_y
                            +
                            Sfx*tr_left_y
                        )
                
            #Bottom right corner coefficients

            elif  k == 2*(nx+2) -2:
                
                #print(str(k) + ": Bottom right corner coefficients")
               
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu)    
                
                #an
                A_y[k,k+nx+2] = -(Sfy/dy)*(2*mu + Lambda)

                #aw                
                A_y[k,k-1] = -(Sfx/dx)*(mu)
                
                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                + U_NE #r
                                                + U_E #r
                                                - U_NW
                                                - U_W
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    - U_N
                                                    - U_NW
                                                    + U_S #b
                                                    + U_SW #b
                                                )
                            +
                            Sfy*tr_bottom_y
                            +
                            Sfx*tr_right_y
                        )
                     
            #Top left corner coefficients
                
            elif  k == (nx+2)*(ny+2) - 2*(nx+2) +1:

                #print(str(k) + ":Top Left corner coefficients")

                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu)    
                
                #as
                A_y[k,k-nx-2] = -(Sfy/dy)*(2*mu + Lambda)

                #ae                
                A_y[k,k+1] = -(Sfx/dx)*(mu)
                
                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                - U_E
                                                - U_SE
                                                + U_W #l
                                                + U_SW #l
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    + U_NE #t
                                                    + U_N #t
                                                    - U_SE
                                                    - U_S
                                                )
                            +
                            Sfy*tr_top_y
                            +
                            Sfx*tr_left_y
                        )    
                
            #Top right corner coefficients
                
            elif  k == (nx+2)*(ny+2) - (nx+2) -2:   

                #print(str(k) + ":Top right corner coefficients")

                #ap
                
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu)    
                
                #as
                A_y[k,k-nx-2] = -(Sfy/dy)*(2*mu + Lambda)

                #aw                
                A_y[k,k-1] = -(Sfx/dx)*(mu)
                
                b_y[k] = (
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                - U_E #r
                                                - U_SE #r
                                                + U_W
                                                + U_SW
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    - U_N #t
                                                    - U_NW #t
                                                    + U_S
                                                    + U_SW
                                                )
                            +
                            Sfy*tr_top_y
                            +
                            Sfx*tr_right_y
                        )
            
            # Center Bottom Boundaries

            elif  (k > nx+3) & (k < 2*(nx+2) -1):

                #print(str(k) + ": Bottom Center Coefficents for Cell")


                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu) + (Sfx/dx)*(mu)

                #an
                A_y[k,k+nx+2] = -(Sfy/dy)*(2*mu + Lambda)
                
                #ae
                A_y[k,k+1] = -(Sfx/dx)*(mu)

                #aw
                A_y[k,k-1] = -(Sfx/dx)*(mu)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(4*dx))*Lambda*(
                                                + U_NE
                                                + U_E
                                                - U_NW
                                                - U_W
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    + U_NE
                                                    + U_N
                                                    - U_SE #b
                                                    - U_S #b
                                                )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    - U_N
                                                    - U_NW
                                                    + U_S #b
                                                    + U_SW #b
                                                )
                            +
                            Sfy*tr_bottom_y
                        )

            # Center Top Boundaries

            elif  ( k > (nx+2)*(ny+2) - 2*(nx+2) +1 ) & ( k < (nx+2)*(ny+2) - (nx+2) -2 ):

                #print(str(k) + ": Top Center Coefficents for Cell")

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*(mu) + (Sfx/dx)*(mu)

                #as
                A_y[k,k-nx-2] = -(Sfy/dy)*(2*mu + Lambda)
                
                #ae
                A_y[k,k+1] = -(Sfx/dx)*(mu)

                #aw
                A_y[k,k-1] = -(Sfx/dx)*(mu)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(4*dx))*Lambda*(
                                                - U_E
                                                - U_SE
                                                + U_W
                                                + U_SW
                                            )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    + U_NE #t
                                                    + U_N #t
                                                    - U_SE
                                                    - U_S
                                                )
                            +
                            (Sfx/(3*dy))*mu*(
                                                    - U_N #t
                                                    - U_NW #t
                                                    + U_S
                                                    + U_SW
                                                )
                            +
                            Sfy*tr_top_y
                        )

            # Center Left Boundaries

            elif  ( k > nx+3) & ( k%(nx+2) == 1 ) & ( k < (nx+2)*(ny+2) - 2*(nx+2) +1):

                #print(str(k) + ": Left Center Coefficents for Cell")

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*mu

                #an
                A_y[k,k+nx+2] = -(Sfy/dy)*(2*mu + Lambda)
                
                #as
                A_y[k,k-nx-2] = -(Sfy/dy)*(2*mu + Lambda)

                #ae
                A_y[k,k+1] = -(Sfx/dx)*(mu)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy) 
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                + U_NE
                                                + U_E
                                                - U_NW #l
                                                - U_W #l
                                            )
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                - U_E
                                                - U_SE
                                                + U_W #l
                                                + U_SW #l 
                                            )
                            +
                            (Sfx/(4*dy))*mu*(
                                                   + U_NE
                                                   + U_N
                                                   - U_SE
                                                   - U_S
                                                )
                            +
                            Sfx*tr_left_y
                        )

            # Center Right Boundaries

            elif  ( k > 2*(nx+2) -2) & ( k%(nx+2) == nx ) & ( k < (nx+2)*(ny+2) - (nx+2) -2 ):

                #print(str(k) + ": Right Center Coefficents for Cell")

                #ap
                A_y[k,k] = (rho*dx*dy/(dt**2)) + (Sfy/dy)*(2*mu + Lambda) + (Sfy/dy)*(2*mu + Lambda) + (Sfx/dx)*mu

                #an
                A_y[k,k+nx+2] = -(Sfy/dy)*(2*mu + Lambda)
                
                #as
                A_y[k,k-nx-2] = -(Sfy/dy)*(2*mu + Lambda)

                #aw
                A_y[k,k-1] = -(Sfx/dx)*(mu)

                b_y[k] =(
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy)
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                + U_NE #r
                                                + U_E #r
                                                - U_NW 
                                                - U_W 
                                            )
                            +
                            (Sfy/(3*dx))*Lambda*(
                                                - U_E #r
                                                - U_SE #r
                                                + U_W 
                                                + U_SW 
                                            )
                            +
                            (Sfx/(4*dy))*mu*(
                                                   - U_N 
                                                   - U_NW
                                                   + U_S
                                                   + U_SW
                                                )
                            +
                            Sfx*tr_right_y
                        )


            #### Points on Boundary ####

            # Center Bottom

            elif (k > 1) & (k < nx):

                #print(str(k) +  ": Bottom points")

                #ap
                A_y[k,k] = (4*mu + 2*Lambda)/dy

                #an
                A_y[k,k+nx+2] = -(4*mu + 2*Lambda)/dy

                #b
                b_y[k] =(
                        tr_bottom_y
                        +
                        (Lambda/(2*dx))*(
                                        + U_E
                                        - U_W
                                    )
                        ) 

            # Center Top

            elif ( k >= ((nx+2)*(ny+2) - nx) ) & ( k < (nx+2)*(ny+2) - 2) :
                
                #print( str(k) + ": Center points" )
                
                #ap
                A_y[k,k] = (4*mu + 2*Lambda)/dy

                #as
                A_y[k,k-nx-2] = - (4*mu + 2*Lambda)/dy

                #b
                b_y[k] =(
                        tr_top_y
                        +
                        (Lambda/(2*dx))*(
                                        - U_E
                                        + U_W
                                    )
                        ) 
                
            # Center Left

            elif ( k > 2*(nx+2) -1) & (k < (ny)*(nx+2) ) & (k%(nx+2) == 0):

                #print( str(k) + ": Left points" )

                #ap
                A_y[k,k] = (2*mu)/dx

                #ae
                A_y[k,k+1] = -(2*mu)/dx

                #b
                b_y[k] =(
                        tr_left_y
                        +
                        (mu/(2*dy))*(
                                        + U_N
                                        - U_S
                                    )
                        )

            # Center Right

            elif ( k > 2*(nx+2) -1) & (k < (ny)*(nx+2)) & (k%(nx+2) == nx +1):

                #print( str(k) + ": Right points" )

                #ap
                A_y[k,k] = (2*mu)/dx

                #aw
                A_y[k,k-1] = -(2*mu)/dx

                #b
                b_y[k] =(
                        tr_right_x
                        +
                        (mu/(2*dy))*(
                                        - U_N
                                        + U_S
                                    )
                        )

            # Bottom Left: Bottom

            elif (k == 1):
                #print( str(k) + ": Bottom Left: Bottom points" )
                #ap
                A_y[k,k] = (4*mu + 2*Lambda)/dx

                #ae
                A_y[k,k+1] = -(4*mu + 2*Lambda)/dx

                #b
                b_y[k] =(
                        tr_bottom_y
                        +
                        ((2*mu)/(3*dx))*(
                                        + U_E
                                        - U_W
                                    )
                        ) 

            # Bottom Left: Left

            elif (k == nx + 2):

                #print( str(k) +  ": Bottom Left: Left points" )
                
                #ap
                A_y[k,k] = (2*mu)/dx

                #ae
                A_y[k,k+1] = -(2*mu)/dx

                #b
                b_y[k] =(
                        tr_left_y
                        +
                        ((2*mu)/(3*dy))*(
                                        + U_N
                                        - U_S
                                    )
                        )
            
            # Bottom Right: Right

            elif ( k == 2*(nx+2) -1) :

                #print( str(k) + ": Bottom Right: Right points" )

                #ap
                A_y[k,k] = (2*mu)/dx

                #aw
                A_y[k,k-1] = -(2*mu)/dx

                #b
                b_y[k] =(
                        tr_right_y
                        +
                        ((2*mu)/(3*dy))*(
                                        - U_N
                                        + U_S
                                    )
                        )

            # Bottom Right: Bottom

            elif (k == nx) : 

                #print( str(k) + ": Bottom Right: Bottom points" )


                #ap
                A_y[k,k] = (4*mu + 2*Lambda)/dy

                #an
                A_y[k,k+nx+2] = -(4*mu + 2*Lambda)/dy

                #b
                b_y[k] =(
                        tr_bottom_y
                        +
                        ((2*Lambda)/(3*dx))*(
                                        + U_E
                                        - U_W
                                    )
                        )

            # Top Left: Left

            elif ( k == (nx+2)*(ny+2) - 2*(nx+2) ) :

                #print( str(k) + " Top Left: Left points" )

                
                #ap
                A_y[k,k] = 2*mu/dx

                #ae
                A_y[k,k+1] = - 2*mu/dx

                #b
                b_y[k] =(
                        tr_left_y
                        +
                        ((2*mu)/(3*dy))*(
                                        + U_N
                                        - U_S
                                    )
                        )

            # Top Left: Top

            elif (k == (nx+2)*(ny+2) - (nx+2) +1):

                #print( str(k) + " Top Left: Top points" )

                
                #ap
                A_y[k,k] = (4*mu + 2*Lambda)/dy

                #as
                A_y[k,k-nx-2] = -(4*mu + 2*Lambda)/dy

                #b
                b_y[k] =(
                        tr_top_y
                        +
                        ((2*Lambda)/(3*dx))*(
                                        - U_E
                                        + U_W
                                    )
                        )

            # Top Right: Right

            elif ( k == (nx+2)*(ny+2) - (nx+2) - 1) :

                #print( str(k) + " Top Right: Right points" )

                #ap
                A_y[k,k] =  2*mu/dx

                #aw
                A_y[k,k-1] = - 2*mu/dx

                #b
                b_y[k] =(
                        tr_right_y
                        +
                        ((2*mu)/(3*dy))*(
                                        - U_N
                                        + U_S
                                    )
                        )

            # Top Right: Top
            
            elif ( k == (nx+2)*(ny+2) - 2 ) : 

                #print( str(k) + " Top Right: Top points" )

                #ap
                A_y[k,k] = (4*Lambda + 2*mu)/dy

                #as
                A_y[k,k-nx-2] = -(4*Lambda + 2*mu)/dy

                #b
                b_y[k] =(
                        tr_top_y
                        +
                        ((2*Lambda)/(3*dx))*(
                                        - U_E
                                        + U_W
                                    )
                        ) 
            # Corner Points

            elif (k == 0):
                #print(str(k) + ": Bottom Left Corner Point on Boundary")

                A_y[k,k] = 1
                b_y[k] = 1e-20

            elif (k == nx + 1):
                #print(str(k) + ": Bottom Right Corner Point on Boundary")
                A_y[k,k] = 1
                b_y[k] = 1e-20

            elif (k == (nx+2)*(ny +1) ): 
                #print(str(k) + ": Top Left Corner Point on Boundary")
                A_y[k,k] = 1
                b_y[k] = 1e-20

            elif (k == (nx+2)*(ny +2) - 1) : 
                #print(str(k) + ": Top Right Corner Point on Boundary")
                A_y[k,k] = 1
                b_y[k] = 1e-20


            else:

                #print(str(k) + ": NO BC Cell")

                b_y[k] = (
                            (rho/(dt**2))*( 2*(u_old[k,1])*dx*dy - u_old_old[k,1]*dx*dy)
                            + (Sfy/(4*dx))*Lambda*(
                                                + U_NE    # + u_ne
                                                - U_NW    # - u_nw
                                                - U_SE    # - u_se
                                                + U_SW    # + u_sw
                                            )
                            +(Sfx/(4*dy))*mu*(
                                                    + U_NE    # + u_ne
                                                    - U_NW    # - u_nw
                                                    - U_SE    # - u_se
                                                    + U_SW    # + u_sw 
                                                )
                            )
        saveArray(str(time) + "/A_y_" + str(icorr) , A_y.todense())   
        saveArray(str(time) + "/b_y_" + str(icorr), b_y)     
               
                
        ## Solve the y-equation
        Uy = spsolve(A_y, b_y)
        Uy = np.array(Uy)

        saveArray(str(time) + "/Uy_" + str(icorr), Uy)
        
        # ## Update u_new with new x and y displacements

        u_new1 = np.vstack((Ux, Uy)).T
        saveArray (str(time) + "/u_new1_" + str(icorr), u_new1)    
        saveArray (str(time) + "/u_previous_" + str(icorr), u_previous)

        # # u_new[:,0] = Ux
        # # u_new[:,1] = Uy
        
        ## Calculate the residual of each iteration
        
        normFactor = np.max(u_new1)
        residual = math.sqrt(np.mean((u_new1 - u_previous)**2))/normFactor
        tolerance = 1e-6

        u_new = u_new1
    
        ##print out residual of each iteration
        print("icorr = {:01d}, Residual = {:.20f}, normFactor = {:.20f}".format(icorr, residual, normFactor))
             
        ## Convergence check
        
        if residual < tolerance:
            
            break
            
        elif icorr > maxcorr:
            
            break
            
        
        icorr = icorr + 1
    
    ## Update variables
    
    u_old_old = u_old
    u_old = u_new
    
    # ## Add the boundary conditions to the solution matrices
    # u_reshape_x = u_new[:,0].reshape(nx, ny)
    # u_reshape_y = u_new[:,1].reshape(nx, ny)
    
    # #Boundary conditions for u matrix
    # U = np.zeros((nx+2,ny+2))
    # U[0,1:nx+1] = u_bottom
    # U[1:ny+1,0] = u_left
    # U[1:ny+1,nx+1] = u_right
    # U[ny+1,1:nx+1] = u_top    
    # U[1:nx+1,1:nx+1] = u_reshape_x
    
    # # #Boundary conditions for v matrix
    # V = np.zeros((nx+2,ny+2))
    # V[0,1:nx+1] = v_bottom
    # V[1:ny+1,0] = v_left
    # V[1:ny+1,nx+1] = v_right
    # V[ny+1,1:nx+1] = v_top    
    # V[1:nx+1,1:nx+1] = u_reshape_y  
    
#     #Modulus of displacement
#     disp_mod = np.sqrt((U**2) + (V**2))
#     disp_mod_converted = disp_mod*1e6 #Convert displacement to micrometers
    
#     # ## Plot the results

#     import plotly.graph_objects as go

#     disp_mod = np.sqrt((U**2) + (V**2))*1e-6

#     dispx, dispy = np.meshgrid(x, y)
#     fig = go.Figure(data=[go.Surface(z=disp_mod_converted, x=dispx, y=dispy)])
    
# fig.show()   



icorr = 1, Residual = 0.53940245360645733275, normFactor = 0.00000000000000806745
icorr = 2, Residual = 84038.14495382446330040693, normFactor = 0.00000000000000111915
icorr = 3, Residual = 0.19877334560410953035, normFactor = 0.00000000356223632608
icorr = 4, Residual = 0.17233523333834940838, normFactor = 0.00002526401380689031
icorr = 5, Residual = 30.78635227921750683322, normFactor = 0.00000106497700541152
icorr = 6, Residual = 14001.35433022404322400689, normFactor = 0.00001439903799224022
icorr = 7, Residual = 0.19876953625889565158, normFactor = 7.63793107964520334718
icorr = 8, Residual = 0.17233523336656825231, normFactor = 54169.71383560285903513432
icorr = 9, Residual = 30.78635207446669852516, normFactor = 2283.46533063538026908645
icorr = 10, Residual = 14001.35432751008738705423, normFactor = 30873.62791901636592228897
icorr = 11, Residual = 0.19876953624860210779, normFactor = 16376833118.80846214294433593750
icorr = 12, Residual = 0.17233523336656295100, normFactor = 1

/tmp/ipykernel_17511/1087346102.py:1447: RuntimeWarning: overflow encountered in square
  residual = math.sqrt(np.mean((u_new1 - u_previous)**2))/normFactor


icorr = 75, Residual = inf, normFactor = 3268112023007897531195033688772501934127775501619276573641820388998516484170945686364788292523601696494866697611293898674361903282749052546144785492846521090048.00000000000000000000
icorr = 76, Residual = inf, normFactor = 23178095079391891755844691501309830250676720363018508362607348447865938298457549068334175079455382023796954843848636435059004284729318900667831951044028671406899200.00000000000000000000
icorr = 77, Residual = inf, normFactor = 977047373582236716646166057866097876388765612844535796649035047376252513840652010992302093232487771301461512283648551704713348960462733354951088123799185433559040.00000000000000000000
icorr = 78, Residual = inf, normFactor = 13210183945048756683986039907142206878035189950614359622915818696373598717905623187040833838917726271218744828826737814450784536563932069155857359281634474517856256.00000000000000000000
icorr = 79, Residual = inf, normFactor = 700730664350915415527720638374960307903416932387552979659

In [133]:
A = A_x

In [134]:
array = A_x - A_y

In [135]:
b_x

mu

780000000000.0